# Generic Monte-Carlo Tree Search for Poetry Generation

This notebook shows how enact's rewind / replay functionality can be used
to search the space of executions of a python program. This allows us
to implement advanced algorithms such as Monte-Carlo Tree Search (MCTS)
in a problem-independent manner: MCTS just becomes a particular way in which
we want to explore the search space of the scaffolding program.

We demonstrate using a line-by-line rhyming poetry generator: Rhyming poetry can
be viewed as a planning problem, since a bad choice for a rhyme word earlier in
the poem can make it difficult to find decent completions. 

We can model line-by-line poetry generation via the following scaffolding program:
```python
def extend_line(poem_so_far: str) -> str:
  ... # Calls LLM to extend the poem by a single line.

def generate_poem() -> str:
  poem = ''
  for _ in range(N_LINES):
    poem = extend_line(poem)
```

Enact allows us to write algorithms that search the space of executions of a
scaffolding program such as `generate_poem`. For example, if we want to generate
different last lines for a single poem, we can do the following:

```python
invocation = enact.invoke(generate_poem)  # Generate initial poem
full_poems = [invocation.get_output()]
for _ in range(N):
  variant = invocation.rewind().replay()  # Reroll the last the extend_line call
  full_poems.append(variant.get_output())
```

The remainder of this notebook shows how enact can be used to implement a generic
Monte-Carlo Tree Search (MCTS) that works for any scaffolding program, based on the
same principle: By directly searching the tree of executions of the scaffolding
program using enact's `rewind` and `replay` functionality.

## MCTS

Monte-carlo tree search (MCTS) is a search algorithm that uses a critic to focus
effort on high-scoring areas of the search space. When applied to line-by-line
poetry generation, MCTS will generate a set of complete seed poems. In each
search step, it will pick a prefix of a previously completed poem and generate
a fresh completion. MCTS will optimistically choose prefixes that do not have
many completions or prefixes that lead to high-scoring completions on average.

![MCTS](mcts.png)

In the (made up) example above, we start with two seed poems, one terrible and one
mediocre. MCTS chooses to regenerate a new last line for the mediocre poem, since it
seems more promising than the terrible poem.

In terms of enact, we can view MCTS as choosing to rewind and replay the invocation
of the second poem.


## Prerequisites and API keys

In [1]:
%pip install enact openai randomname tqdm numpy --quiet


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import asyncio
import re
from typing import Awaitable

import enact       # Journaled executions
import openai      # GPT API
import randomname  # Extra randomness for prompt.
import tqdm        # progress bar

import api_keys    # API keys


# Read from file or env variable.
API_KEY = api_keys.OPENAI_API_KEY.get()


## Defining a line-by-line poetry generator

We first define our scaffolding program: a line-by-line poetry generator which
autoregressively calls GPT in a for-loop.

In [2]:
@enact.register
class Failed(enact.ExceptionResource):
  """Raised when a subtask fails."""


@enact.register  # Track this function's execution.
async def gpt(prompt: str, temperature: float=1.0, model='gpt-4o') -> str:
  """Call the GPT completion API"""
  client = openai.AsyncClient(api_key=API_KEY)
  response = await client.chat.completions.create(
    model=model,
    messages=[{'role': 'system', 'content': prompt}],
    temperature=temperature)
  return response.choices[0].message.content


@enact.register
async def extend_poem(
    poem: str, prompt: str, lines: int,
    temperature: float = 1.0) -> str:
  """Extend the poem by one line."""
  line_index = len(poem.strip().split('\n'))
  gpt_prompt = f'''
We are writing a poem for the prompt: {prompt}.

This is the poem so far:

{poem if poem else '<NEW POEM>'}

Extend the poem by one single line. Respond only with the new line and say
nothing else. This will be line nr {line_index + 1} out of a total of {lines}
lines.

Avoid cliches.

Respond only with one line.

Ignore the following random seed words:
{', '.join(randomname.generate('nouns/') for _ in range(10))}

Be sure to respond only with one line of poetry and nothing else.
'''
  result = await gpt(gpt_prompt, temperature)
  lines = result.strip().split('\n')
  if len(lines) != 1:
    raise Failed('Responded with more than one line')
  line: str = lines[0]
  line = line.strip('"\'').strip()
  return poem + f'\n{line}'

@enact.register
async def generate_poem(prompt: str, lines: int) -> str:
  """Generate a poem line by line."""
  poem = ''
  for _ in range(lines):
    poem = await extend_poem(poem, prompt, lines)
  return poem


Our poetry generator can be run as follows, although it may sometimes fail,
e.g., when GPT produces output in a wrong format.

In [4]:
try:
  two_line_poem = await generate_poem(
    'a two line rhyming poem in the style of Alexander Pope', lines=2)
  print(two_line_poem)
except Exception as e:
  print(f'Failed with "{e}". Try again.')


The heavens mock man's schemes with silent grace.
With folly's pride, he runs a fruitless race.


We can also inspect the call trace by generating an invocation, this
requires us to work in the context of a store:

In [4]:
with enact.InMemoryStore() as store:
  invocation = await enact.invoke_async(generate_poem, ('a poem about the sea', 2))
  print(enact.invocation_summary(invocation, show_input=False, show_output=False))
  print(invocation.get_output())

-><function generate_poem at 0x7f28ad8f6ca0>(...) 
  -><function extend_poem at 0x7f28ad8f6700>(...) 
    -><function gpt at 0x7f28ad8f6040>(...) 
  -><function extend_poem at 0x7f28ad8f6700>(...) 
    -><function gpt at 0x7f28ad8f6040>(...) 

Dunes of whispers echo where waves undress the shore,
Seafoam fingers linger, tracing secrets once stored.


We can regenerate the last line by rewinding and replaying the invocation.

In [5]:
with store:
  invocation = await invocation.rewind().replay_async()
  print(invocation.get_output())


Dunes of whispers echo where waves undress the shore,
Seagulls sketch their stories on boundless blue canvas skies.


## Define a critic

We would like to search the space of possible executions of our `generate_poem`
function. In order to do so, we need to define a scoring function that can
assign a numeric value to generated poems. We use another call to GPT for this.

In [5]:
@enact.register
async def score_poem(poem: str) -> float:
  gpt_prompt: str = f'''
Score the following poem on a scale from 0 to 10, where 0 is the worst poem and
10 is the best. Ask yourself the following questions:
- Does it have good rhythm?
- Does it have natural-sounding rhymes?
- Is the poem self-contained and complete?
- Does the poem use of cliches?
- Does the poem use akward phrasings or forced rhymes?
- Does the poem use overelaborate / sophomoric language?

Respond in the following format:

<very short summary of your critique>
SCORE: <score>

This is the poem:
{poem}
'''
  score = await gpt(gpt_prompt, 1.1)
  return float(re.search(r'SCORE: ([\d\.]+)', score).group(1))


We can apply our poetry critic to the poem we generated earlier:

In [6]:
try:
  print(two_line_poem)
  print(f'Score: {await score_poem(two_line_poem)}')
except Exception as e:
  print(f'Try again. Scoring failed with: {e}')


The heavens mock man's schemes with silent grace.
With folly's pride, he runs a fruitless race.
Score: 8.0


## Implementing Monte Carlo Tree Search

Enact can rewind and replay executions of registered python functions. We can
use this to explore the space of possible program executions using Monte Carlo
Tree Search.

In [7]:
import dataclasses
from typing import Callable, Dict, List, Optional
import numpy as np


# Increase this number for more exploration
EXPLORATION_AMOUNT = 3
# Increase this to explore more from the root node,
# i.e., to create more new poems from scratch.
EXPLORE_ROOT_NODE = 1
# Conservative quality prior to add for bootstrapped LCB. This penalizes
# poems that we haven't thouroughly explored yet.
LCB_PRIOR = [1.0]
# Sample node with highest UCB at this temperature. If set to 0, always picks
# the max UCB node.
SAMPLING_TEMPERATURE=0.25

# How many parallel calls.
PARALLELISM=5


@enact.register
@dataclasses.dataclass
class Node(enact.Resource):
  invocation: enact.Invocation  # A (possibly-partial) execution.
  parent: Optional['Node']      # The parent node.
  scores: List[float]           # List of scores for this subtree.

  @property
  def tree_ucb(self) -> float:
    """Computes a heuristic UCB on scores for tree search."""
    if not self.parent:
      num_parent_scores = EXPLORE_ROOT_NODE * len(self.scores)
    else:
      num_parent_scores = len(self.parent.scores)
    return np.mean(self.scores) + EXPLORATION_AMOUNT * np.sqrt(
      np.log(1e-2 + num_parent_scores) / len(self.scores))

  @property
  def lcb(self) -> float:
    """Computes a lower confidence bound (30th percentile)."""
    scores = self.scores
    scores = scores + LCB_PRIOR

    return  np.percentile(
      np.mean(np.random.choice(
        scores, (10000, len(scores))), axis=1),
      30.0)


@enact.register
@dataclasses.dataclass
class MCTS(enact.Resource):
  """A generic implementation for MCTS over abitrary scaffolding programs."""
  scorer: Callable[[str], Awaitable[float]]
  nodes: Dict[str, Node] = dataclasses.field(
    default_factory=dict)

  async def add(
      self,
      invocation: enact.Invocation):
    """Add an invocation to the MCTS tree."""
    child = None
    score = await self.compute_score(invocation)
    # Propagate scores up the invocation tree.
    while True:
      cur = enact.commit(invocation)
      node = self.nodes.setdefault(cur.id, Node(invocation, None, []))
      node.scores.append(score)
      if child:
        child.parent = node
      child = node
      if not invocation.response().children:
        break
      invocation = invocation.rewind()  # Generate execution prefix.
    return score

  def next_node(self):
    """Select next node according to highest UCB."""
    ucbs = np.array([node.tree_ucb for node in self.nodes.values()])
    if not SAMPLING_TEMPERATURE:
      idx = np.argmax(ucbs)
    else:
      softmax_ps = np.exp(ucbs / SAMPLING_TEMPERATURE) / np.sum(
        np.exp(ucbs / SAMPLING_TEMPERATURE))
      idx = np.random.choice(len(ucbs), p=softmax_ps)
    return list(self.nodes.values())[idx]

  async def compute_score(self, invocation: enact.Invocation) -> float:
    """Assign a score to the invocation."""
    if not invocation.successful():
      return -1.0
    try:
      return await self.scorer(invocation.get_output())
    except enact.InputRequest:
      raise
    except:
      raise Failed

  @enact.register
  async def step(self):
    """Run one step of MCTS and return generated output and score."""
    assert self.nodes, 'Please call add before step.'
    node = self.next_node()
    if node.invocation.response().children:
      print(f'\nContinuing partial invocation:\n{node.invocation.response().children[-1]().get_output()}')
    else:
      print(f'\nCreating new root invocation')
    invocation = await self.replay(node.invocation)
    score = await self.add(invocation)
    if invocation.successful():
      print(f'== New invocation:\n{invocation.get_output()}')
      print(f'== Score: {score}')
      print()
      return invocation.get_output(), score

  @enact.register
  async def invoke(self, fun: Callable, args=(), kwargs=None):
    """Wrap invocation in a registered function, since it is non-deterministic."""
    return await enact.invoke_async(fun, args=args, kwargs=kwargs)

  @enact.register
  async def replay(self, invocation: enact.Invocation) -> enact.Invocation:
    """Wrap replay in a registered function, since it is non-deterministic."""
    return await invocation.replay_async()

  @enact.register
  async def add_seed(self, fun, args=(), kwargs=None):
    """Add a seed invocation. Return success."""
    invocation = await self.invoke(fun, args=args, kwargs=kwargs)
    if not invocation.successful():
      return False
    try:
      score = await self.add(invocation)
      print(f'== Added seed invocation:\n{invocation.get_output()}')
      print(f'== Score: {score}')
      print()
    except Failed:
      return False
    return True

  @enact.register
  async def run(self, fun, args=(), kwargs=None, seeds=5, iterations=100):
    """Run MCTS for a given number of iterations."""
    print('Creating seed poems')
    await asyncio.gather(
      *[self.add_seed(fun, args=args, kwargs=kwargs)
        for _ in range(seeds)])

    if not self.nodes:
      raise Failed('Could not find a successful seed invocation.')

    current_tasks: List[asyncio.Task] = []
    for _ in tqdm.trange(max(iterations - PARALLELISM, 1), desc='Running MCTS'):
      current_tasks = [t for t in current_tasks if not t.done()]
      while len(current_tasks) < PARALLELISM:
        current_tasks.append(asyncio.create_task(self.step()))
      done, _ = await asyncio.wait(current_tasks, return_when=asyncio.FIRST_COMPLETED)
      for d in done:
        try:
          await d
        except Failed:
          continue
    for t in current_tasks:
      try:
        await t
      except Failed:
        continue

  def highest_lcb(self):
    """Return the highest LCB output."""
    success_nodes = [node for node in self.nodes.values() if node.invocation.successful()]
    if not success_nodes:
      return None
    return max(success_nodes, key=lambda node: node.lcb).invocation.get_output()

  def highest_mean(self):
    """Return the output with the higest mean score."""
    success_nodes = [node for node in self.nodes.values() if node.invocation.successful()]
    if not success_nodes:
      return None
    return max(success_nodes, key=lambda node: np.mean(node.scores)).invocation.get_output()

  def most_visited(self):
    """Return the output that was visited the most."""
    success_nodes = [node for node in self.nodes.values() if node.invocation.successful()]
    if not success_nodes:
      return None
    return max(success_nodes, key=lambda node: len(node.scores)).invocation.get_output()

The above mostly a straightforward implementation of MCTS, but it is worth highlighting
that each node in the search tree is associated with a partial execution of the program
in the form of an enact `Invocation`.

```python
class Node(enact.Resource):
  invocation: enact.Invocation  # A (possibly-partial) execution.
  ...

```

In each step of the MCTS search, a tree node is picked to be extended, which is accomplished
via `node.invocation.replay()`. This results in a completed invocation that extends
`node.invocation`.

## Running MCTS on our poetry generator

Note that our implementation of MCTS is *generic* in the sense that it accepts
an arbitrary enact-registered python function and scorer. Instead of
having to implement a domain-specific search tree where the nodes correspond to
partial poems or chess-boards, we can search directly in the space of program
executions regardless of the domain in which the underlying program operates. 
Thus, MCTS can be used as a general purpose refinement-operator that improves
the quality of a generative program using a scoring function.

Enact requires a `Store` object which records inputs, outputs and call-traces
in content addressable memory.

In [8]:
store = enact.InMemoryStore()

We can now run MCTS for a set number of iterations by providing it with
the program `generate_poem`, whose execution tree we'd like to search,
and our automatic scoring function `score_poem`, which allows us to
associate a quality score with outputs.

In [10]:
with store:
  mcts = MCTS(score_poem)
  await mcts.run(
    generate_poem,
    args=('A short rhyming poem about the human condition using simple language', 4),
    seeds=10,
    iterations=100)

Creating seed poems
== Added seed invocation:

Hearts that soar yet stumble too,
Dreams collide in skies of blue,
Tears that water seeds of pain,
Hope emerges through the rain.
== Score: 8.0

== Added seed invocation:

Hearts that yearn for joy anew,
Eyes that seek what's real and true,
Hands that work and minds that grow,
Souls that wander through the blue.
== Score: 7.0

== Added seed invocation:

People laugh and people cry,
Some will question, others sigh,
Dreams are born and then they die,
Life moves on, like clouds that fly.
== Score: 6.0

== Added seed invocation:

Sometimes smiles hide the deepest fears,
While tears can speak what no one hears,
Dreams may hold our highest hopes dear,
In laughter's warmth, we all persevere.
== Score: 8.0

== Added seed invocation:

We laugh, we cry, we seek, we strive,
We dance through storms, yet stay alive,
Our fragile bonds through darkness shine,
In fleeting moments, souls entwine.
== Score: 9.0

== Added seed invocation:

We laugh and we cr

Running MCTS:   0%|          | 0/95 [00:00<?, ?it/s]


Continuing partial invocation:

We laugh, we cry, we seek, we strive,

Continuing partial invocation:

We laugh, we cry, we seek, we strive,

Continuing partial invocation:

We laugh, we cry, we seek, we strive,

Continuing partial invocation:

We laugh, we cry, we seek, we strive,

Continuing partial invocation:

We laugh, we cry, we seek, we strive,


Running MCTS:   1%|          | 1/95 [00:03<05:37,  3.59s/it]

== New invocation:

We laugh, we cry, we seek, we strive,
We dream of more, just to survive,
In tangled paths, we learn to cope,
We hold on tight, with threads of hope.
== Score: 8.0


Continuing partial invocation:

Hearts that soar yet stumble too,
== New invocation:

We laugh, we cry, we seek, we strive,
We share our dreams, we fight to thrive,
We build our houses, walk our roads,
Beneath the stars, our stories unfold.
== Score: 8.0


Continuing partial invocation:

We laugh, we cry, we seek, we strive,
We dance through storms, yet stay alive,


...

Running MCTS:  99%|█████████▉| 94/95 [01:00<00:00,  2.01it/s]

== New invocation:

We laugh and we cry under endless skies,
We seek simple truths, and live woven lies,
Our hearts beat in rhythm, yet question the ties,
In silent surrender, the sunset replies.
== Score: 8.0


Continuing partial invocation:

Tears fall unnoticed in the rain,
As laughter echoes through the pain,
Yet hearts beat on, despite the strain,


Running MCTS: 100%|██████████| 95/95 [01:01<00:00,  1.56it/s]

== New invocation:

We laugh and we cry under endless skies,
We seek simple truths, and live woven lies,
We dance through the questions that none can define,
In the heart’s quiet corners, we search for a sign.
== Score: 9.0



== New invocation:

We laugh and we cry under endless skies,
We seek simple truths, and live woven lies,
Our hearts dance in rhythm to life's ebbing sighs,
In the beauty and frailty of life, we rise.
== Score: 8.0

== New invocation:

We laugh and we cry under endless skies,
We seek simple truths, and live woven lies,
We dream and we doubt, in the world we devise,
We dance with the stars, yet fear our own ties.
== Score: 8.0

== New invocation:

Tears fall unnoticed in the rain,
As laughter echoes through the pain,
Hearts will mend and break again,
Hope blooms brighter after strife.
== Score: 7.0

== New invocation:

Tears fall unnoticed in the rain,
As laughter echoes through the pain,
Yet hearts beat on, despite the strain,
Hope arises, like a small flame.
== Score: 7.0



We now print the best results, estimated in different ways.

In [11]:
with store:
  print(f'\nMost visited:\n{mcts.most_visited()}')
  print(f'\nHighest mean score:\n{mcts.highest_mean()}')
  print(f'\nHighest LCB:\n{mcts.highest_lcb()}')


Most visited:

Tears fall unnoticed in the rain,
Faces hide their silent pain,
Hearts yearn for warmth, but break in vain,
As we seek the light, shadows remain.

Highest mean score:

We laugh, we cry, we seek, we strive,
We dance through storms, yet stay alive,
Our fragile bonds through darkness shine,
In fleeting moments, souls entwine.

Highest LCB:

Tears fall unnoticed in the rain,
Faces hide their silent pain,
Hearts yearn for warmth, but break in vain,
As we seek the light, shadows remain.
